In [2]:

import argparse
import json
import os
import pickle

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn import preprocessing
from torch.utils.data import DataLoader
from torchvision import transforms



In [3]:
import pandas as pd
BRACS = pd.read_excel (r'C:/Users/nerea/Documents/Master 2022/TFM/BRACS.xlsx')

In [4]:
BRACS.head()

,WSI Filename,Patient Id,RoI,WSI label,Set
0,BRACS_264,85,24,PB,Testing
1,BRACS_265,87,18,UDH,Validation
2,BRACS_280,30,19,IC,Training
3,BRACS_281,111,37,IC,Training
4,BRACS_283,109,49,IC,Training


In [5]:
BRACS.replace({"Testing": "test", "Training": "train", "Validation":"val"}, inplace=True)
clases=['N', 'PB', 'UDH', 'FEA', 'ADH', 'DCIS', 'IC']
BRACS['number']=0
BRACS['Group']=0
for i in range(len(clases)):
  BRACS.loc[BRACS['WSI label']==clases[i],['number']] = i
BRACS.loc[BRACS['WSI label'].isin(('N', 'PB', 'UDH')),['Group']] = 'BT'
BRACS.loc[BRACS['WSI label'].isin(('FEA', 'ADH')),['Group']] = 'AT'
BRACS.loc[BRACS['WSI label'].isin(('DCIS','IC')),['Group']] = 'MT'
BRACS['number']=BRACS['number'].astype(str)

In [6]:
BRACS.head()

,WSI Filename,Patient Id,RoI,WSI label,Set,number,Group
0,BRACS_264,85,24,PB,test,1,BT
1,BRACS_265,87,18,UDH,val,2,BT
2,BRACS_280,30,19,IC,train,6,MT
3,BRACS_281,111,37,IC,train,6,MT
4,BRACS_283,109,49,IC,train,6,MT



# Obtención de paths de WSI a partir del excel



In [8]:
datasets=['train','test','val']
ohe = preprocessing.OneHotEncoder(sparse=False)
classes = np.array(clases)
ohe.fit(classes.reshape(-1,1))

datasets=['train','test','val']
data_WSI = {}
data_WSI['train'] = {'x': [], 'y': []}
data_WSI['val'] = {'x': [], 'y': []}
data_WSI['test'] = {'x': [], 'y': []}
paths={}
for i in datasets:
  aux=BRACS[BRACS['Set']==i]
  files_WSI='/home/pc-i1-2/BRACS_WSI/'+i+'/'+'Group_'+aux['Group']+'/Type_'+aux['WSI label']+'/'+aux['WSI Filename']+'.svs'
  aux=BRACS[BRACS['Set']==i]
  files_WSI='/home/pc-i1-2/BRACS_WSI/'+i+'/'+'Group_'+aux['Group']+'/Type_'+aux['WSI label']+'/'+aux['WSI Filename']+'.svs'
  files_WSI=files_WSI.tolist()
  paths[i]=files_WSI
  data_WSI[i]['y'] = aux['WSI label']
  data_WSI[i]['x'] = files_WSI
  data_WSI[i]['x'] = np.asarray(data_WSI[i]['x'])
  data_WSI[i]['y'] = np.asarray(data_WSI[i]['y'])
  data_WSI[i]['y'] = ohe.transform(data_WSI[i]['y'].reshape(-1,1))
data_WSI['val']

{'x': array(['/home/pc-i1-2/BRACS_WSI/val/Group_BT/Type_UDH/BRACS_265.svs',
        '/home/pc-i1-2/BRACS_WSI/val/Group_MT/Type_DCIS/BRACS_735.svs',
        '/home/pc-i1-2/BRACS_WSI/val/Group_MT/Type_DCIS/BRACS_736.svs',
        '/home/pc-i1-2/BRACS_WSI/val/Group_MT/Type_DCIS/BRACS_737.svs',
        '/home/pc-i1-2/BRACS_WSI/val/Group_MT/Type_IC/BRACS_738.svs',
        '/home/pc-i1-2/BRACS_WSI/val/Group_MT/Type_IC/BRACS_739.svs',
        '/home/pc-i1-2/BRACS_WSI/val/Group_MT/Type_IC/BRACS_740.svs',
        '/home/pc-i1-2/BRACS_WSI/val/Group_BT/Type_N/BRACS_1261.svs',
        '/home/pc-i1-2/BRACS_WSI/val/Group_AT/Type_ADH/BRACS_1271.svs',
        '/home/pc-i1-2/BRACS_WSI/val/Group_BT/Type_PB/BRACS_1275.svs',
        '/home/pc-i1-2/BRACS_WSI/val/Group_BT/Type_PB/BRACS_1276.svs',
        '/home/pc-i1-2/BRACS_WSI/val/Group_BT/Type_UDH/BRACS_1295.svs',
        '/home/pc-i1-2/BRACS_WSI/val/Group_BT/Type_N/BRACS_1296.svs',
        '/home/pc-i1-2/BRACS_WSI/val/Group_MT/Type_IC/BRACS_1361.svs',
 

# Obtención de los paths de cada muestra de región de interés (sin clases)

In [9]:
import glob
import numpy as np
from sklearn import preprocessing


clases_roi=pd.Series(['0_N', '1_PB', '2_UDH', '3_FEA', '4_ADH', '5_DCIS', '6_IC'])

data_RoI = {}
data_RoI['train'] = {'x': [], 'y': []}
data_RoI['val'] = {'x': [], 'y': []}
data_RoI['test'] = {'x': [], 'y': []}
files_RoI=list()
for i in datasets:
  paths_RoI='C:\\Users\\nerea\\Desktop\\TFM\\BRACS_RoI\\lastest_version\\'+i+'\\'+clases_roi+'\\'
  for j in range(6):
    aux=glob.glob(paths_RoI[j]+'*.png')
    files_RoI+=aux





# Creación de patches

In [10]:

"""Script to extract and save patches."""
from cv2 import imread, imwrite
from pathlib import Path
from tqdm import tqdm
import numpy as np 
import os

import errno


paths = []
labels = []
log_file = open('log_svs2png.txt', 'w')
patch_size = 512

for filename in files_RoI:
    f = str(filename)
    save_name = f.split('\\')[-1].split('.')[0]
    save_path = f.split('_')[0]+'_RoI_patches'+'\\'+'\\'.join(f.split('\\')[6:-1])
    #crea los directorios si no existen
    os.makedirs(save_path, exist_ok=True)
    im = imread(f)
    nobgr_img_blocks = []
    for j in range(0,im.shape[1],patch_size):
        for i in range(0,im.shape[0],patch_size):
            block = im[i:i+patch_size, j:j+patch_size]
            if block.shape == (patch_size, patch_size, 3):
                if(np.mean(block[:][:][0]) < 220.0 and np.mean(block[:][:][1]) < 220.0
                and np.mean(block[:][:][2]) < 220.0):
                    nobgr_img_blocks.append(block)
    index = 0
    for block in nobgr_img_blocks:
        if not os.path.isfile(save_path +'\\'+ save_name + '_' + str(index) + '.jpeg'): 
            try:
                imwrite(save_path +'\\'+ save_name + '_' + str(index) + '.jpeg', block)
            except:
                log_file.write(f+'\n') 
                continue
        else:
            print('already converted')
        
        index += 1
log_file.close()


already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already converted
already co

# "Image_reader" a mano (sin usar las clases)

In [229]:
import glob
import numpy as np
from sklearn import preprocessing

datasets=['train','test','val']
ohe = preprocessing.OneHotEncoder(sparse=False)
classes = np.array(clases)
ohe.fit(classes.reshape(-1,1))
clases_roi=pd.Series(['0_N', '1_PB', '2_UDH', '3_FEA', '4_ADH', '5_DCIS', '6_IC'])

data_RoI = {}
data_RoI['train'] = {'x': [], 'y': []}
data_RoI['val'] = {'x': [], 'y': []}
data_RoI['test'] = {'x': [], 'y': []}
files_RoI_pat=list()
for i in datasets:
  paths_RoI_pat='C:\\Users\\nerea\\Desktop\\TFM\\BRACS_RoI_patches\\lastest_version\\'+i+'\\'+clases_roi+'\\'
  for j in range(6):
    aux=glob.glob(paths_RoI_pat[j]+'*.jpeg')
    files_RoI_pat+=aux
    data_RoI[i]['x'].extend(aux)  
  print(files_RoI_pat)
  label=[]
  for k in files_RoI_pat:
    aux=k.split('\\')[-2].split('_')[-1]
    label.append(aux)
  #label
  data_RoI[i]['y'].extend(label)
  data_RoI[i]['x'] = np.asarray(data_RoI[i]['x'])
  data_RoI[i]['y'] = np.asarray(data_RoI[i]['y'])
  data_RoI[i]['y'] = ohe.transform(data_RoI[i]['y'].reshape(-1,1))


data_RoI

['C:\\Users\\nerea\\Desktop\\TFM\\BRACS_RoI_patches\\lastest_version\\train\\0_N\\BRACS_1286_N_40_0.jpeg', 'C:\\Users\\nerea\\Desktop\\TFM\\BRACS_RoI_patches\\lastest_version\\train\\0_N\\BRACS_1286_N_40_1.jpeg', 'C:\\Users\\nerea\\Desktop\\TFM\\BRACS_RoI_patches\\lastest_version\\train\\0_N\\BRACS_1286_N_40_2.jpeg', 'C:\\Users\\nerea\\Desktop\\TFM\\BRACS_RoI_patches\\lastest_version\\train\\0_N\\BRACS_1286_N_40_3.jpeg', 'C:\\Users\\nerea\\Desktop\\TFM\\BRACS_RoI_patches\\lastest_version\\train\\0_N\\BRACS_1286_N_40_4.jpeg', 'C:\\Users\\nerea\\Desktop\\TFM\\BRACS_RoI_patches\\lastest_version\\train\\0_N\\BRACS_1286_N_40_5.jpeg', 'C:\\Users\\nerea\\Desktop\\TFM\\BRACS_RoI_patches\\lastest_version\\train\\0_N\\BRACS_1286_N_40_6.jpeg', 'C:\\Users\\nerea\\Desktop\\TFM\\BRACS_RoI_patches\\lastest_version\\train\\0_N\\BRACS_1286_N_40_7.jpeg', 'C:\\Users\\nerea\\Desktop\\TFM\\BRACS_RoI_patches\\lastest_version\\train\\0_N\\BRACS_1286_N_40_8.jpeg', 'C:\\Users\\nerea\\Desktop\\TFM\\BRACS_RoI_pa

{'train': {'x': array(['C:\\Users\\nerea\\Desktop\\TFM\\BRACS_RoI_patches\\lastest_version\\train\\0_N\\BRACS_1286_N_40_0.jpeg',
         'C:\\Users\\nerea\\Desktop\\TFM\\BRACS_RoI_patches\\lastest_version\\train\\0_N\\BRACS_1286_N_40_1.jpeg',
         'C:\\Users\\nerea\\Desktop\\TFM\\BRACS_RoI_patches\\lastest_version\\train\\0_N\\BRACS_1286_N_40_2.jpeg',
         'C:\\Users\\nerea\\Desktop\\TFM\\BRACS_RoI_patches\\lastest_version\\train\\0_N\\BRACS_1286_N_40_3.jpeg',
         'C:\\Users\\nerea\\Desktop\\TFM\\BRACS_RoI_patches\\lastest_version\\train\\0_N\\BRACS_1286_N_40_4.jpeg',
         'C:\\Users\\nerea\\Desktop\\TFM\\BRACS_RoI_patches\\lastest_version\\train\\0_N\\BRACS_1286_N_40_5.jpeg',
         'C:\\Users\\nerea\\Desktop\\TFM\\BRACS_RoI_patches\\lastest_version\\train\\0_N\\BRACS_1286_N_40_6.jpeg',
         'C:\\Users\\nerea\\Desktop\\TFM\\BRACS_RoI_patches\\lastest_version\\train\\0_N\\BRACS_1286_N_40_7.jpeg',
         'C:\\Users\\nerea\\Desktop\\TFM\\BRACS_RoI_patches\\laste

# Clase Dataset: transforma los datos para poder usar pytorch

In [230]:
import torch
from torch.utils import data
import numpy as np
from torchvision import transforms
from cv2 import imread
import pandas as pd

class Dataset(dataset):
    'Characterizes a dataset for PyTorch'
    def __init__(self, inputs, labels, transform):
        'Initialization'
        self.labels = labels
        self.inputs = inputs
        self.transform = transform
    def __len__(self):
        'Denotes the total number of samples'
        return len(self.inputs)
        
    def __getitem__(self, index):
        'Generates one sample of data'
    
        file = self.inputs[index]
        x = imread(file).astype(np.uint8)

        if self.transform:
            x = self.transform(transforms.ToPILImage()(x))
        
        y = self.labels[index]
        
        y = torch.from_numpy(np.asarray(y)).float()
   
        return x, y, file

class TestDataset(dataset):
    'Characterizes the test dataset'
    def __init__(self, inputs, labels, transform):
        'Initialization'
        self.labels = labels
        self.inputs = inputs
        self.transform = transform
    
    def __len__(self):
        'Denotes the total number of samples'
        return len(self.inputs)
        
    def __getitem__(self, index):
        'Generates one sample of data'
    
        files = self.inputs[index]
        images = []
        for f in files:
            x = imread(f).astype(np.uint8)
            if self.transform:
               x = self.transform(transforms.ToPILImage()(x)) 
            images.append(x)
            
        y = self.labels[index]
        y = torch.from_numpy(np.asarray(y)).float()
   
        return images, y, files

class TestDatasetImgGen(dataset):
    'Characterizes the test dataset'
    def __init__(self, inputs, labels, transform, num_variables):
        'Initialization'
        self.labels = labels
        self.inputs = inputs
        self.transform = transform
        self.num_variables = num_variables

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.inputs)
        
    def __getitem__(self, index):
        'Generates one sample of data'
        files = self.inputs[index][0]
        file_gen = self.inputs[index][1]
        images = []
        case_ids = []

        # read img files
        for f in files:
            x = imread(f).astype(np.uint8)
            if self.transform:
               x = self.transform(transforms.ToPILImage()(x)) 
            images.append(x)
            case_ids.append(f)
 
        # read gen file
        data = pd.read_csv(file_gen, sep=',')
        try:
            gen = data.values[0, 1:self.num_variables+1].astype('float32')
        except:
            gen = data.values[0, 2:self.num_variables+2].astype('float32')
        
        gen = torch.from_numpy(gen).float()
        y = self.labels[index]
        y = torch.from_numpy(np.asarray(y)).float()
   
        return images, gen, y, case_ids

class PatchDataset(dataset):
    'Characterizes a dataset for PyTorch'
    def __init__(self, inputs, labels, scaler, case_ids):
        'Initialization'
        self.inputs = inputs
        self.labels = labels
        self.scaler = scaler
        self.case_ids = case_ids

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.inputs)
        
    def __getitem__(self, index):
        'Generates one sample of data'
        x = self.scaler.transform(self.inputs[index].reshape(1, -1))
        x = torch.from_numpy(x).float()
        
        return x[0], self.labels[index][0], self.case_ids[index]

class TestPatchDataset(dataset):
    'Characterizes a dataset for PyTorch'
    def __init__(self, inputs, labels, scaler, case_ids, len_sizes):
        'Initialization'
        self.inputs = inputs
        self.labels = labels
        self.scaler = scaler
        self.case_ids = case_ids
        self.len_size = len_sizes
        self.index0 = 0
        self.index1 = len_sizes[0]
    def __len__(self):
        'Denotes the total number of samples'
        return len(self.case_ids)
        
    def __getitem__(self, index):
        'Generates one sample of data'

        x = self.inputs[self.index0:self.index1]
        y = self.labels[self.index0]

        x = torch.from_numpy(x).float()
        self.index0 = self.index1
        if index+1 < len(self.len_size):
            self.index1 = self.index0+self.len_size[index+1]
        
        return x, y, self.case_ids[index]

In [ ]:
def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
        m.bias.fill_(0.01)

# Clase train_pred

In [231]:
import time
import torch
import copy
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
import optuna
from collections import Counter 
import logging
from sklearn.metrics import roc_auc_score, confusion_matrix
import torch.nn.functional as F
import numpy as np
from sklearn.cluster import KMeans

device = torch.device("cuda:0" if torch.cuda.is_available()
                                   else "cpu")

def train_model(model, criterion, optimizer, dataloaders, dataset_sizes,
                lr_scheduler, save_path, name, num_epochs=25, verbose=True):
    LOG = save_path + "/execution-" + name + ".log" 
    logging.basicConfig(filename=LOG, filemode="w", level=logging.DEBUG)  

    # console handler  
    console = logging.StreamHandler()  
    console.setLevel(logging.ERROR)  
    logging.getLogger("").addHandler(console)

    logger = logging.getLogger(__name__)
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_epoch = 0
    best_auc = 0

    acc_array = {'train': [], 'val': []}
    loss_array = {'train': [], 'val': []}
    auc_array = {'train': [], 'val': []}
    best_val_preds = []
    best_train_preds = []
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        logger.debug('Epoch {}/{}'.format(epoch, num_epochs - 1))
        logger.debug('-' * 10)
        val_preds = []
        train_preds = []
        train_case_ids = []
        train_labels = []
        val_labels = []
        train_labels_auc = []
        train_preds_auc = []
        val_labels_auc = []
        val_preds_auc = []
        sizes = {'train': 0, 'val': 0}
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels, case_ids in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                inputs.requires_grad = True
                labels = labels.to(device)
                # zero the parameter gradients
                optimizer.zero_grad()
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    _, mlabels = torch.max(labels, 1)

                    loss = criterion(outputs, mlabels)
                    if phase == 'val':
                        val_preds += list(preds.cpu().numpy())
                        val_labels += list(mlabels.cpu().numpy())
                        val_labels_auc += [labels.cpu().numpy()]
                        val_preds_auc += [preds.cpu().numpy()]

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        train_preds += list(preds.cpu().numpy())
                        train_case_ids += [case_ids]
                        train_labels += list(mlabels.cpu().numpy())
                        train_labels_auc += [labels.cpu().numpy()]
                        train_preds_auc += [preds.cpu().numpy()]
                        loss.backward()
                        optimizer.step()
                
                # statistics

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == mlabels)
                sizes[phase] += inputs.size(0)
                
            epoch_loss = running_loss / sizes[phase]
            epoch_acc = running_corrects.item() / sizes[phase]

            loss_array[phase].append(epoch_loss)
            acc_array[phase].append(epoch_acc)
            
            if verbose:
                print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                    phase, epoch_loss, epoch_acc))
                logger.debug('{} Loss: {:.4f} Acc: {:.4f}'.format(
                    phase, epoch_loss, epoch_acc))

            if phase == 'val' and epoch_acc > best_acc:
                best_epoch = epoch
                best_acc = epoch_acc
                #best_auc = epoch_auc
                best_model_wts = copy.deepcopy(model.state_dict())
                best_val_preds = val_preds[:]
                best_train_preds = train_preds[:]
        print()

    time_elapsed = time.time() - since
    if verbose:
        print('Training complete in {:.0f}m {:.0f}s'.format(
            time_elapsed // 60, time_elapsed % 60))
        print('Best val Acc: {:4f}'.format(best_acc))

        logger.debug('Training complete in {:.0f}m {:.0f}s'.format(
            time_elapsed // 60, time_elapsed % 60))
        logger.debug('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    if best_val_preds == []:
        best_val_preds = val_preds[:]
    results = {
        'model': model,
        'best_acc': best_acc,
        #'best_auc': best_auc,
        'best_epoch': best_epoch,
        'acc_array': acc_array,
        'loss_array': loss_array,
        'auc_array': auc_array,
        'val_preds': best_val_preds,
        'val_labels': val_labels,
        'train_preds': best_train_preds,
        'train_case_ids': train_case_ids,
        'train_labels': train_labels
    }

    return results

def predict_patches(model, dataloader, dataset_size):
    from sklearn.metrics import accuracy_score
    model.eval()
    val_preds = []
    val_labels = []

    for inputs, labels, caseids in dataloader:
        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            _, mlabels = torch.max(labels, 1)
            
            val_preds += list(preds.cpu().numpy())
            val_labels += list(mlabels.cpu().numpy())
    
    acc = accuracy_score(val_labels, val_preds)

    print('Accuracy {:.4f}'.format(acc))

    return acc, val_preds, val_labels
    
def predict_WSI(model, dataloader, dataset_size, verbose=True):
        """Predict an image by using patches."""
        activation = {}
        def _get_features(name):
            def hook(model, input, output):
                activation[name] = input[0].detach()
            return hook

        model.eval()
        since = time.time()
        corrects = 0
        # results variables
        preds = []
        patch_preds = []
        probs = {'N': [], 'PB': [], 'UDH': [],'FEA':[],'ADH':[],'DCIS':[],'IC':[]}
        model.fc.register_forward_hook(_get_features('fc'))
        features = []
        case_ids = []
        # a prediction is going to be made for each patch,
        # and the final prediction will be the most predicted class
        sizes = 0
        i = 0
        patch_labels = []
        global_labels = []
        for inputs, labels, cids in tqdm(dataloader):
            _, mlabel = torch.max(labels, 1)
            predictions = []
            local_features = []
            for patch, cid in zip(inputs, cids):
                patch = patch.to(device).float() 
                output = model(patch)
                _, pred = torch.max(output, 1)
                predictions.append(pred.item())
                local_features.append(activation['fc'].cpu().numpy())
                case_ids.append(cid)
                patch_labels.append(mlabel.item())

            # obtaining final prediction
            final_preds = Counter(predictions)           
            if mlabel.item() == final_preds.most_common(1)[0][0]:
                corrects += 1
            
            #saving preds, labels, features
            preds.append(final_preds.most_common(1)[0][0])
            patch_preds.append(predictions)
            global_labels.append(mlabel.item())
            features.append(local_features)
            
            # saving probabilities
            if 0 in final_preds.keys():
                probs['N'].append(final_preds[0]/len(inputs))
            else:
                probs['N'].append(0)
            if 1 in final_preds.keys():
                probs['PB'].append(final_preds[1]/len(inputs))
            else:
                probs['PB'].append(0)
            if 2 in final_preds.keys():
                probs['UDH'].append(final_preds[2]/len(inputs))
            else:
                probs['UDH'].append(0)
            if 3 in final_preds.keys():
                probs['FEA'].append(final_preds[3]/len(inputs))
            else:
                probs['FEA'].append(0)
            if 4 in final_preds.keys():
                probs['ADH'].append(final_preds[4]/len(inputs))
            else:
                probs['ADF'].append(0)
            if 5 in final_preds.keys():
                probs['DCIS'].append(final_preds[5]/len(inputs))
            else:
                probs['DCIS'].append(0)
            if 6 in final_preds.keys():
                probs['IC'].append(final_preds[6]/len(inputs))
            else:
                probs['IC'].append(0)
            
            del inputs, labels


        acc = corrects / dataset_size

        time_elapsed = time.time() - since
        if verbose:
            print('Test complete in {:.0f}m {:.0f}s'.format(
                  time_elapsed // 60, time_elapsed % 60))
        
        all_results = {
            'acc': acc,
            'preds': preds,
            'patch_preds': patch_preds,
            'patch_labels': patch_labels,
            'global_labels': global_labels,
            'probs': probs,
            'patch_case_ids': case_ids,
            'features': features
        }

        return all_results

def predict_WSI_gen(model, dataloader, dataset_size, verbose=True):
        """Predict an image by using patches."""
        since = time.time()
        corrects = 0
        test_preds = []
        model.eval()
        sizes = 0
        i = 0
        test_labels = []
        with torch.set_grad_enabled(False):
            for inputs, labels, case_ids in tqdm(dataloader):
                predictions = []
                for patch in inputs[0]:
                    patch = patch.to(device).float() 
                    output = model.fc(patch)
                    _, pred = output.max(0)
                    predictions.append(pred.item())
                final_pred = Counter(predictions).most_common(1)[0][0]

                if labels.item() == final_pred:
                    corrects += 1
                
                test_preds.append(final_pred)
                test_labels.append(labels.item())
                
                del inputs, labels
        acc = corrects / dataset_size

        time_elapsed = time.time() - since
        if verbose:
            print('Test complete in {:.0f}m {:.0f}s'.format(
                  time_elapsed // 60, time_elapsed % 60)) 
        return acc, test_preds

def obtain_WSI_features(model, dataloader, method='gap'):
        """Obtain features from patches.
        Args:
            batch_X (np.ndarray): array of numpy arrays of data
        """
        activation = {}
        def _get_features(name):
            def hook(model, input, output):
                activation[name] = input[0].detach()
            return hook

        model.eval()
        model.fc.register_forward_hook(_get_features('fc'))

        features_img = []
        features_gen = []

        for inputs, gen, labels in tqdm(dataloader):
            _, mlabel = torch.max(labels, 1)
            local_features = torch.Tensor().to(device)

            for patch in inputs:
                patch = patch.to(device).float() 
                output = model(patch)
                img_features = torch.cat([local_features, activation['fc']], dim=0)

            # global average pooling
            if method == 'gap':
                img_features = img_features.view(1, img_features.shape[1], 1, img_features.shape[0])
                gap = img_features.mean([2, 3])
                features_img.append(gap.cpu().numpy())
                features_gen.append(gen)
            # using kmeans to obtain three centroids and use those features.
            elif method == 'clusters':
                kmeans = KMeans(n_clusters=3, random_state=0).fit(img_features)
                centroids = torch.from_numpy(kmeans.cluster_centers_.flatten()).to(device)
                combined_features = torch.cat([gen.to(device), centroids], dim=1)
                features = torch.cat([features, combined_features], dim=0)
            
            del inputs, labels, img_features

        return features_img, features_gen

def obtain_patch_gen_features(model, dataloader):
        """Obtain features from patches and gene expression.
        """
        activation = {}
        def _get_features(name):
            def hook(model, input, output):
                activation[name] = input[0].detach()
            return hook

        model.eval()
        model.fc.register_forward_hook(_get_features('fc'))

        features_img = []
        features_gen = []
        labels_all = []
        case_ids =  []

        for inputs, gen, labels, c_id in tqdm(dataloader):
            _, mlabel = torch.max(labels, 1)
            local_features = torch.Tensor().to(device)
            case_ids += [c_id]

            for patch in inputs:
                patch = patch.to(device).float() 
                output = model(patch)
                features_img.append(activation['fc'].cpu().numpy())
                features_gen.append(gen)
                labels_all.append(mlabel.cpu().numpy())
           
            del inputs

        return features_img, features_gen, labels_all, case_ids

def get_probs(model, dataloader, verbose=True):
        model.eval()
        test_probs = {'N': [], 'PB': [], 'UDH': [],'FEA':[],'ADH':[],'DCIS':[],'IC':[]}
        all_labels = []
        with torch.set_grad_enabled(False):
            for inputs, labels, _ in tqdm(dataloader):
                predictions = []
                for patch in inputs[0]:
                    patch = patch.to(device).float() 
                    output = model.fc(patch)
                    out_prob = torch.nn.functional.softmax(output, dim=0)
                    _, pred = out_prob.max(0)
                    predictions.append(pred.item())
                
                final_preds = Counter(predictions)
                # saving probabilities
                if 0 in final_preds.keys():
                    test_probs['N'].append(final_preds[0]/len(inputs))
                else:
                    test_probs['N'].append(0)
                if 1 in final_preds.keys():
                    test_probs['PB'].append(final_preds[1]/len(inputs))
                else:
                    test_probs['PB'].append(0)
                if 2 in final_preds.keys():
                    test_probs['UDH'].append(final_preds[2]/len(inputs))
                else:
                    test_probs['UDH'].append(0)
                if 3 in final_preds.keys():
                    test_probs['FEA'].append(final_preds[3]/len(inputs))
                else:
                    test_probs['FEA'].append(0)
                if 4 in final_preds.keys():
                    test_probs['ADH'].append(final_preds[4]/len(inputs))
                else:
                    test_probs['ADF'].append(0)
                if 5 in final_preds.keys():
                    test_probs['DCIS'].append(final_preds[5]/len(inputs))
                else:
                    test_probs['DCIS'].append(0)
                if 6 in final_preds.keys():
                    test_probs['IC'].append(final_preds[6]/len(inputs))
                else:
                    test_probs['IC'].append(0)
                del inputs, labels
        
        return test_probs, all_labels

# Entrenamiento del modelo: 
- Aun no se han realizado todas las modificaciones, primero se quiere probar sin validación cruzada con unas pocas imagenes de prueba en local
- Falta cambiar la configuración y más adaptaciones a mi problema

In [ ]:
'''
# Parsing arguments by config file
parser = argparse.ArgumentParser(description='Arg parser')
parser.add_argument('--c', action="store", dest="config", type=str,
                    default='config_default.json')
args = parser.parse_args()
config_name = args.config
with open(config_name, 'r') as f:
    config = json.load(f)

# Config information
splits_folder = config['splits_path']
splits = config['n_splits']
batch_size = config['batch_size']
lr = config['lr']
model_name = config['model_name']
epochs = config['epochs']
bool_lr_scheduler = config['lr_scheduler']
results_folder_name = config['results_folder_name']


# Creating folder for saving weights
if not os.path.isdir('weights/'+results_folder_name):
    os.mkdir('weights/'+results_folder_name)

# Creating folder for saving results
if not os.path.isdir('results/'+results_folder_name):
os.mkdir('results/'+results_folder_name)

""" Create readers """
dataReaders = {}
dataReaders['CNN'] = data_RoI

""" Get paths """
# Dataset paths
datasets = ['train', 'val','test']

# Shuffle train set
index_train = np.random.randint(0, len(dataReaders['CNN']['train']['x']), len(dataReaders['CNN']['train']['x']))
dataReaders['CNN']['train']['x'] = dataReaders['CNN']['train']['x'][index_train]
dataReaders['CNN']['train']['y'] = dataReaders['CNN']['train']['y'][index_train]

# create train dataloader
train_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

val_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

dataset_train = Dataset(dataReaders['CNN']['train']['x'], 
                dataReaders['CNN']['train']['y'], train_transform)
dataloader_train = DataLoader(dataset_train, batch_size=32,
                                shuffle=True, num_workers=8, 
                                pin_memory=True)

dataset_val = Dataset(dataReaders['CNN']['val']['x'], 
                dataReaders['CNN']['val']['y'], val_transform)

dataloader_val = DataLoader(dataset_val, batch_size=32,
                            shuffle=False, num_workers=8, 
                            pin_memory=True)

dataset_test = TestDataset(dataReaders['CNN']['test']['x'],
                    dataReaders['CNN']['test']['y'], val_transform)
dataloader_test = DataLoader(dataset_test, batch_size=1,
                                shuffle=False, num_workers=1,
                                pin_memory=True)

dataloaders = {'train': dataloader_train, 'val': dataloader_val, 'test': dataloader_test}
dataset_sizes = {x: len(dataReaders['CNN'][x]['x']) for x in ['train', 'val', 'test']}

model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=True)
clases=['N', 'PB', 'UDH', 'FEA', 'ADH', 'DCIS', 'IC']
for param in model.parameters():
    param.requires_grad = False

num_ftrs = model.fc.in_features

model.fc =  nn.Sequential(
                nn.Dropout(p=0.5),  
                nn.Linear(num_ftrs, len(clases))
            )

# initializing weights with xavier
model.fc.apply(init_weights)
for param in model.fc.parameters():
    param.requires_grad = True
for param in model.layer3.parameters():
    param.requires_grad = True

model = model.to(device)

criterion = nn.CrossEntropyLoss()

# saving results to path
save_path = 'results/'+results_folder_name+'/'

params = [p for p in model.parameters() if p.requires_grad]
optimizer = optim.Adam(params, lr=lr)
if bool_lr_scheduler:
    lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

results = train_model(model=model, criterion=criterion, optimizer=optimizer, dataloaders=dataloaders, 
            dataset_sizes=dataset_sizes, lr_scheduler=lr_scheduler, save_path=save_path, num_epochs=epochs, 
            name=str(i), verbose=True)

print('Best acc in split {}: {}'.format(i, results['best_acc']))

print("Saving model's weights to folder")
torch.save(results['model'].state_dict(), 'weights/'+results_folder_name+'/final_weights_split'+str(i)+'.pkl')

# write dict results on file
a_file = open(save_path+results_folder_name+'_split'+str(i)+'_results_Epoch_'+str(results['best_epoch'])+'.pkl', "wb")
pickle.dump(results, a_file)
a_file.close()

model = results['model']

model.eval()
# save val and train preds

# train

data = pd.DataFrame()
data['Case_Ids'] = results['train_case_ids'][0]
data['Preds'] = results['train_preds']
data['Real'] = results['train_labels']
data.to_excel(save_path+results_folder_name+'_split'+str(i)+'_train_Epoch'+str(results['best_epoch'])+'.xlsx')

# val
val_case_ids = dataReaders['CNN']['val']['x']
val_labels = np.argmax(dataReaders['CNN']['val']['y'], axis=1)

data = pd.DataFrame()
data['Case_Ids'] = val_case_ids
data['Preds'] = results['val_preds']
data['Real'] = val_labels
data.to_excel(save_path+results_folder_name+'_split'+str(i)+'_val_Epoch'+str(results['best_epoch'])+'.xlsx')

# predict on test set  
test_results = predict_WSI(model, dataloader_test, dataset_sizes['test'])
print('Test acc: {:.4f}\n'.format(test_results['acc']))

# write dict results on file
a_file = open(save_path+results_folder_name+'_split'+str(i)+'_test_results.pkl', "wb")
pickle.dump(test_results, a_file)
a_file.close()

test_labels = np.argmax(dataReaders['CNN']['test']['y'], axis=1)
case_ids_test = [x[0].split('/')[3] for x in dataReaders['CNN']['test']['x']]

data = pd.DataFrame()
data['Case_Ids'] = case_ids_test
data['Preds'] = test_results['preds']
data['Prob N'] = test_results['probs']['N']
data['Prob PB'] = test_results['probs']['PB']
data['Prob UDH'] = test_results['probs']['UDH']
data['Prob FEA'] = test_results['probs']['FEA']
data['Prob ADH'] = test_results['probs']['ADH']
data['Prob DCIS'] = test_results['probs']['DCIS']
data['Prob IC'] = test_results['probs']['IC']

data['Real'] = test_results['global_labels']
data.to_excel(save_path+results_folder_name+'_split'+str(i)+'_test.xlsx')

# TRAIN SLIDE
dataReaders = {}
dataReaders['CNN_slide'] = data_RoI

dataset_train_slide = TestDataset(dataReaders['CNN_slide']['train']['x'],
                    dataReaders['CNN_slide']['train']['y'], val_transform)
dataloader_train_slide = DataLoader(dataset_train_slide, batch_size=1,
                                shuffle=False, num_workers=1,
                                pin_memory=True)

train_size = len(dataReaders['CNN_slide']['train']['x'])
train_results = predict_WSI(model, dataloader_train_slide, train_size)

# write dict results on file
a_file = open(save_path+results_folder_name+'_split'+str(i)+'_train_slide_results.pkl', "wb")
pickle.dump(train_results, a_file)
a_file.close()

train_labels = np.argmax(dataReaders['CNN_slide']['train']['y'], axis=1)
case_ids_train = [x[0].split('/')[3] for x in dataReaders['CNN_slide']['train']['x']]

data = pd.DataFrame()
data['Case_Ids'] = case_ids_train
data['Preds'] = train_results['preds']
data['Case_Ids'] = case_ids_test
data['Preds'] = test_results['preds']
data['Prob N'] = test_results['probs']['N']
data['Prob PB'] = test_results['probs']['PB']
data['Prob UDH'] = test_results['probs']['UDH']
data['Prob FEA'] = test_results['probs']['FEA']
data['Prob ADH'] = test_results['probs']['ADH']
data['Prob DCIS'] = test_results['probs']['DCIS']
data['Prob IC'] = test_results['probs']['IC']
data['Real'] = train_results['global_labels']
data.to_excel(save_path+results_folder_name+'_split'+str(i)+'_train_slide.xlsx')
'''